In [9]:
# Importação das bibliotecas necessárias
import numpy as np

# 1. Lê o arquivo e pega as dimensões 

In [5]:
with open('./instances/la01.txt') as file:
    lines = file.read()

dimensions = lines.split('\n')[0]
data = lines.split('\n')[1:]

tasks = dimensions.split()[0]
machines = dimensions.split()[1]
print('Número de tasks: ', tasks)
print('Número de máquinas: ', machines)
print('Dados')
print(data)
print(len(data))

Número de tasks:  10
Número de máquinas:  5
Dados
[' 1 21 0 53 4 95 3 55 2 34', ' 0 21 3 52 4 16 2 26 1 71', ' 3 39 4 98 1 42 2 31 0 12', ' 1 77 0 55 4 79 2 66 3 77', ' 0 83 3 34 2 64 1 19 4 37', ' 1 54 2 43 4 79 0 92 3 62', ' 3 69 4 77 1 87 2 87 0 93', ' 2 38 0 60 1 41 3 24 4 83', ' 3 17 1 49 4 25 0 44 2 98', ' 4 77 3 79 2 43 1 75 0 96']
10


In [75]:
# Transformando os dados
def juntaParMaquinaTempoExecução(vetor):
    resultado = []
    tamanho = len(vetor)
    i = 0
    while i < tamanho:
        resultado.append([vetor[i],vetor[i+1]])
        i = i + 2
    return resultado

# Converte o array em uma matriz
dadosFormatados = []
for item in data:
    dadosFormatados.append(juntaParMaquinaTempoExecução(item.split()))

# Printa para ver o resultado
for i in range(0, len(dadosFormatados)):
    print("Tarefa {}: {}".format(i+1, dadosFormatados[i]))

print("")
print("Exemplo para facilitar o entendimento")
print("Tarefa {}: {}".format(1, dadosFormatados[0]))
for i in range(0, len(dadosFormatados[0])):
    print("Tarefa {} -> Operação 1,{}: {} -> Máquina {} com tempo {}".format(i+1, i+1, dadosFormatados[0][i], dadosFormatados[0][i][0], dadosFormatados[0][i][1]))


Tarefa 1: [['1', '21'], ['0', '53'], ['4', '95'], ['3', '55'], ['2', '34']]
Tarefa 2: [['0', '21'], ['3', '52'], ['4', '16'], ['2', '26'], ['1', '71']]
Tarefa 3: [['3', '39'], ['4', '98'], ['1', '42'], ['2', '31'], ['0', '12']]
Tarefa 4: [['1', '77'], ['0', '55'], ['4', '79'], ['2', '66'], ['3', '77']]
Tarefa 5: [['0', '83'], ['3', '34'], ['2', '64'], ['1', '19'], ['4', '37']]
Tarefa 6: [['1', '54'], ['2', '43'], ['4', '79'], ['0', '92'], ['3', '62']]
Tarefa 7: [['3', '69'], ['4', '77'], ['1', '87'], ['2', '87'], ['0', '93']]
Tarefa 8: [['2', '38'], ['0', '60'], ['1', '41'], ['3', '24'], ['4', '83']]
Tarefa 9: [['3', '17'], ['1', '49'], ['4', '25'], ['0', '44'], ['2', '98']]
Tarefa 10: [['4', '77'], ['3', '79'], ['2', '43'], ['1', '75'], ['0', '96']]

Exemplo para facilitar o entendimento
Tarefa 1: [['1', '21'], ['0', '53'], ['4', '95'], ['3', '55'], ['2', '34']]
Tarefa 1 -> Operação 1,1: ['1', '21'] -> Máquina 1 com tempo 21
Tarefa 2 -> Operação 1,2: ['0', '53'] -> Máquina 0 com tempo

In [40]:
# Dados mockados para teste
posicaoCandidataTeste = [['1','1'], ['3', '1'], ['2', '1'], ['3', '2'], ['1', '2'], ['2', '2'], ['3', '3'], ['2', '3'], ['1', '3']]
matrizOperacoesTempoTeste = [[['1', '2'], ['2', '3'], ['3', '1']], [['2', '3'], ['1', '1'], ['3', '2']], [['2', '2'], ['3', '3'], ['1', '2']]]


'2'

In [74]:
# Função de aptidão
def funcaoAptidao(posicaoCandidata, matrizOperacoesTempo, numeroMaquinas, numeroTarefas):
    vetorTempoMaquinasEmExecucao = np.zeros(numeroMaquinas)
    vetorTarefasEmExecucao = np.zeros(numeroTarefas)
    # i[0] -> TAREFA
    # i[1] -> OPERACAO
    for i in posicaoCandidata:
        print(i)
        maquina = int(matrizOperacoesTempo[int(i[0]) - 1 ][int(i[1]) - 1][0])
        tempoMaquina = int(matrizOperacoesTempo[int(i[0]) - 1][int(i[1]) - 1][1])
        tempoAtualAuxiliar = max(vetorTempoMaquinasEmExecucao[maquina - 1], vetorTarefasEmExecucao[int(i[0]) - 1])
        vetorTempoMaquinasEmExecucao[maquina - 1] = tempoAtualAuxiliar + tempoMaquina
        vetorTarefasEmExecucao[int(i[0]) - 1] = max(vetorTempoMaquinasEmExecucao[maquina - 1], vetorTarefasEmExecucao[int(i[0]) - 1])
        print("vetorTempoMaquinasEmExecucao: {}".format(vetorTempoMaquinasEmExecucao))
        print("")
    return [max(vetorTempoMaquinasEmExecucao), np.argmax(vetorTempoMaquinasEmExecucao) + 1]

funcaoAptidao(posicaoCandidataTeste, matrizOperacoesTempoTeste, 3, 3)

['1', '1']
vetorTempoMaquinasEmExecucao: [2. 0. 0.]

['3', '1']
vetorTempoMaquinasEmExecucao: [2. 2. 0.]

['2', '1']
vetorTempoMaquinasEmExecucao: [2. 5. 0.]

['3', '2']
vetorTempoMaquinasEmExecucao: [2. 5. 5.]

['1', '2']
vetorTempoMaquinasEmExecucao: [2. 8. 5.]

['2', '2']
vetorTempoMaquinasEmExecucao: [6. 8. 5.]

['3', '3']
vetorTempoMaquinasEmExecucao: [8. 8. 5.]

['2', '3']
vetorTempoMaquinasEmExecucao: [8. 8. 8.]

['1', '3']
vetorTempoMaquinasEmExecucao: [8. 8. 9.]



[9.0, 3]